In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os.path
import re
import sys
import tarfile

import numpy as np
from six.moves import urllib
import tensorflow as tf

import glob
import os
import numpy as np
import matplotlib.pyplot as plt

import cv2
import math

DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

# Image Recognition

In [ ]:
class NodeLookup(object):
  """Converts integer node ID's to human readable labels."""

  def __init__(self,
               label_lookup_path=None,
               uid_lookup_path=None):
    if not label_lookup_path:
        label_lookup_path = 'model/imagenet_2012_challenge_label_map_proto.pbtxt'
    if not uid_lookup_path: 
        uid_lookup_path = 'model/imagenet_synset_to_human_label_map.txt'
    
    self.node_lookup = self.load(label_lookup_path, uid_lookup_path)

  def load(self, label_lookup_path, uid_lookup_path):
    """Loads a human readable English name for each softmax node.

    Args:
      label_lookup_path: string UID to integer node ID.
      uid_lookup_path: string UID to human-readable string.

    Returns:
      dict from integer node ID to human-readable string.
    """
    if not tf.gfile.Exists(uid_lookup_path):
      tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not tf.gfile.Exists(label_lookup_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = tf.gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
      parsed_items = p.findall(line)
      uid = parsed_items[0]
      human_string = parsed_items[2]
      uid_to_human[uid] = human_string

    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = tf.gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
      if line.startswith('  target_class:'):
        target_class = int(line.split(': ')[1])
      if line.startswith('  target_class_string:'):
        target_class_string = line.split(': ')[1]
        node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
      if val not in uid_to_human:
        tf.logging.fatal('Failed to locate: %s', val)
      name = uid_to_human[val]
      node_id_to_name[key] = name

    return node_id_to_name

  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]


def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile('model/classify_image_graph_def.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

def run_inference_on_image_file(images, num_top_predictions=3):
    """Runs inference on an image.

    Args:
    image: Image file name.

    Returns:
    Nothing
    """
    for image in images :
        if not tf.gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)
    
    images_data = {}
    for image in images :
        image_data = tf.gfile.FastGFile(image, 'rb').read()
        images_data[image] = image_data
        
    result = {}
    # Creates graph from saved GraphDef.
    create_graph()

    with tf.Session() as sess:
        for filename in images_data :
            image_data = images_data[filename]            
            # Some useful tensors:
            # 'softmax:0': A tensor containing the normalized prediction across
            #   1000 labels.
            # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
            #   float description of the image.
            # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
            #   encoding of the image.
            # Runs the softmax tensor by feeding the image_data as input to the graph.
            softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
            predictions = sess.run(softmax_tensor,
                                   {'DecodeJpeg/contents:0': image_data})
            predictions = np.squeeze(predictions)

            # Creates node ID --> English string lookup.
            node_lookup = NodeLookup()
            file_writer = tf.summary.FileWriter('/home/izza/Project/DataExtraction', sess.graph)

            prediction_summary = "";
            top_k = predictions.argsort()[-num_top_predictions:][::-1]
            for node_id in top_k:
                human_string = node_lookup.id_to_string(node_id)
                score = predictions[node_id]
                prediction_summary += '%s (score = %.5f)' % (human_string, score)
            result[filename] = prediction_summary
    return result

def run_inference_on_image(images_data, num_top_predictions=3):
    result = {}
    # Creates graph from saved GraphDef.
    create_graph()

    with tf.Session() as sess:
        for filename in images_data :
            image_data = images_data[filename]            
            softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
            x = tf.constant(image_data)
            predictions = sess.run(softmax_tensor,
                                   {'DecodeJpeg:0': image_data})
            predictions = np.squeeze(predictions)

            # Creates node ID --> English string lookup.
            node_lookup = NodeLookup()
#             file_writer = tf.summary.FileWriter('/home/izza/Project/DataExtraction', sess.graph)

            prediction_summary = "";
            top_k = predictions.argsort()[-num_top_predictions:][::-1]
            for node_id in top_k:
                human_string = node_lookup.id_to_string(node_id)
                score = predictions[node_id]
                prediction_summary += '%s (score = %.5f)' % (human_string, score)
            result[filename] = prediction_summary
    return result

def maybe_download_and_extract():
  """Download and extract model tar file."""
  dest_directory = "model"
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)

def recognize_object_from_file(image_files):
  maybe_download_and_extract()
  return run_inference_on_image_file(image_files)

def recognize_object(images):
    maybe_download_and_extract()
    return run_inference_on_image(images)

In [ ]:
recognize_object_from_file(["images/fish.png"])

# Video Slicing

In [ ]:
def slice_video(videoFile = 'videos/big_buck_bunny.mp4', frameDiffTime=1, save=False, video_name="unknown") : 
    cap = cv2.VideoCapture(videoFile)
    frameRate = frameDiffTime * cap.get(5) #frame rate
    frames = {}
    
    count = 0
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        
        if (frameId % math.floor(frameRate) == 0):
            filename = "videos/frames/" +  video_name + "_" + str(count) + ".jpg"
            frames[filename] = frame
            
            if (save) :
                cv2.imwrite(filename, frame)
        count += 1
    cap.release()
    return frames

In [ ]:
frames = slice_video("videos/animal.mp4", 30)
print("extracted " + str(len(frames)) + " images")

In [ ]:
predictions = recognize_object(frames)
for key in sorted(frames):
    plt.imshow(frames[key], cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()
    print(predictions[key])

# Prediction with Saved File

In [ ]:
frames = slice_video("videos/big_buck_bunny.mp4", True, "bunny")
print("extracted " + str(len(frames)) + " images")

predictions = recognize_object_from_file(frames.keys())
for key in sorted(frames):
    plt.imshow(frames[key], cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()
    print(predictions[key])